In [89]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
import pandas as pd
import numpy as np
import joblib

# Compute engine stand alone

In [91]:
file = joblib.load('schemas.barfi')

In [92]:
from_client = joblib.load('_from_client.pkl')

In [93]:
from blocks import init
from streamlit import session_state as session
session.dados = pd.DataFrame()
init()

starting blocks
colums: RangeIndex(start=0, stop=0, step=1)


In [94]:
from Barfi import ComputeEngine

def get_execution(fluxo, base_blocks_list):
    _ce = ComputeEngine(blocks=base_blocks_list)
    _ce.add_editor_state(fluxo)
    _ce._map_block_link()
    _ce._execute_compute()

In [95]:
get_execution(file['soma3'], session.blocks)

Resultado: 3


# Entrada block

In [241]:
file = joblib.load('schemas.barfi')
entr = file['entradas']

In [242]:
get_execution(entr, session.blocks)

nome: entrada-1 valor: 1
nome: entrada-2 valor: 2
Resultado: 3


In [243]:
def get_dict_from_interfaces(interface):
    aux = {}
    for i in interface:
        aux[i[0]] = i[1]
    return aux

def encode_dict_to_interface(aux_dict):
    interface = [[key, aux_dict[key]] for key in aux_dict]
    return interface

def encode_val_to_entradas(entra, val_dict):
    aux = entra.copy()
    for entrada in aux:
        name = entrada['name']
        tmp = get_dict_from_interfaces(entrada['interfaces'])
        tmp['entrada']['value'] = val_dict[name]
        encoded_interface = encode_dict_to_interface(tmp)
        entrada['interfaces'] = encoded_interface
    return aux

def get_ids_entradas(entradas):
    aux = entradas.copy()
    ids = []
    for entrada in aux:
        tmp = get_dict_from_interfaces(entrada['interfaces'])
        ids.append(tmp['entrada']['id'])
    return ids

def get_ids_from_conn(connections, to_ids):
    aux = connections.copy()
    entry_dict = {}
    ids = []
    ids_from = []
    pos = []
    for i, conn in enumerate(connections):
        if conn['to'] in to_ids:
            entry_dict[i] = {'id': conn['id'],
                             'from': conn['from'],
                             'to': conn['to']}
    return entry_dict

In [244]:
def encode_entradas_to_const(entra, val_dict):
    aux = entra['nodes'].copy()
    for entrada in aux:
        name = entrada['name']
        if name in val_dict:
            entrada['type'] = 'constante'
            entrada['options'] = [['valor', val_dict[name]]]
            tmp = get_dict_from_interfaces(entrada['interfaces'])
            del tmp['entrada']
            encoded_interface = encode_dict_to_interface(tmp)
            entrada['interfaces'] = encoded_interface

In [245]:
def get_to_dict_from_conn(connection):
    aux = {i['to']:i for i in connection}
    return aux

def encode_conn_from_dict(dict_conn):
    return list(dict_conn.values())

In [246]:
def delete_connections(fluxo, to_ids):
    tmp = get_to_dict_from_conn(fluxo['connections'])
    for i in to_ids:
        try:
            del tmp[i]
        except:
            pass
    return tmp

In [247]:
def get_fluxo_const(fluxo, val_dict):
    saidas = [i for i in fluxo['nodes'] if i['type'] == 'saida']
    saida_names = [i['name'] for i in saidas]
    entradas = [i for i in fluxo['nodes'] if i['type'] == 'entrada']
    entrada_names = [i['name'] for i in entradas]
    ids_entradas = get_ids_entradas(entradas)
    entry_conn = get_ids_from_conn(fluxo['connections'], ids_entradas)
    tmp = delete_connections(fluxo, ids_entradas)
    fluxo['connections'] = encode_conn_from_dict(tmp)
    encode_entradas_to_const(fluxo, val_dict)

In [248]:
new_entr = entr.copy()

In [249]:
val_dict = {'entrada-1': 2, 'entrada-2': 3}
# tmp = entradas.copy()
# new_entr = encode_val_to_entradas(tmp, val_dict)

In [250]:
get_fluxo_const(entr, val_dict)

In [251]:
get_execution(entr, session.blocks)

Resultado: 5


In [235]:
ids_entradas = get_ids_entradas(entradas)
entry_conn = get_ids_from_conn(entr['connections'], ids_entradas)

In [236]:
tmp = delete_connections(new_entr, ids_entradas)

In [237]:
new_entr['connections'] = encode_conn_from_dict(tmp)

In [238]:
encode_entradas_to_const(new_entr, val_dict)

In [239]:
get_execution(new_entr, session.blocks)

Resultado: 5
